<a href="https://colab.research.google.com/github/sksizer/dat490/blob/main/BFRSS_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment Setup
- check env
- set and test paths for data

In [48]:
import os
from IPython import get_ipython
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)

def is_colab():
    return 'google.colab' in str(get_ipython())

if is_colab() and not os.path.exists('/content/drive'):
    from google.colab import drive
    drive.mount('/content/drive')

if is_colab():
    BFRSS_DATA_PATH = '/content/drive/MyDrive/DAT490/data/LLCP2023.parquet'
    BFRSS_CODEBOOK_PATH = '/content/drive/MyDrive/DAT490/data/codebook_USCODE23_LLCP_021924.HTML'
else:
    BFRSS_CODEBOOK_PATH = './data/'
    BFRSS_DATA_PATH = './data/'

if not os.path.exists(BFRSS_DATA_PATH):
    raise Exception(f"Data path ${BFRSS_DATA_PATH} does not exist")

if not os.path.exists(BFRSS_CODEBOOK_PATH):
    raise Exception(f"Codebook path ${BFRSS_CODEBOOK_PATH} does not exist")
logger.info('Environment setup complete')


INFO:root:Environment setup complete


# Load Data and Metadata
- creates starting DF `bfrss_raw_df` from BFRSS data
- extract metadata: parses [Codebook](https://github.com/sksizer/dat490/blob/ main/data/codebook_USCODE23_LLCP_021924.HTML) into a dictionary that uses columns as keys:
  ```
  bfrss_metadata
  # to get the metadata for a column:
  bfrss_metadata['COLUMN1']

  # It also has value to value descriptions such as:
  bfrss_metadata['COLUMN1'].value_lookup[1] # will return something like 'Number of times worked out in last week'
  ```

In [36]:
import pandas as pd
bfrss_raw_df = pd.read_parquet(BFRSS_DATA_PATH)
bfrss_raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 433323 entries, 0 to 433322
Columns: 350 entries, _STATE to _DRNKDRV
dtypes: float64(345), object(5)
memory usage: 1.1+ GB


In [38]:
from pydantic import BaseModel, Field
from typing import Optional
import re
from pathlib import Path
from typing import Dict
from bs4 import BeautifulSoup, PageElement


class ColumnMetadata(BaseModel):
    computed: bool
    label: str
    sas_variable_name: str
    section_name: Optional[str] = None
    section_number: Optional[int] = None
    question_number: Optional[int] = None
    column: Optional[str] = None  # Can be a range like "1-2" or single number
    type_of_variable: Optional[str] = None  # "Num" or "Char"
    question_prologue: Optional[str] = None
    question: Optional[str] = None
    value_lookup: dict[None | int, str] # This is a dictionary that returns the textual


def get_value_lookup(table:PageElement) -> Dict[None | int, str]:
    """
    Given one of the branch table objects, we can extract out in a fairly
    simple manner all the possible values for the target column

    Simplified table structure example:
    <table>
    <tbody>
    <tr>
    <td>value</td> (which might be a single int value, blank or could be a range
    <td>Value description
    </tr>
    </tbody>
    </table>

    :param table:
    :return:
    """
    value_dict : Dict[None | int, str] = {} # Stores the value to value description

    for tr in table.find('tbody').find_all('tr'):
        cells = tr.find_all('td')
        if len(cells) < 2:
            continue

        value_text = cells[0].text.strip()
        description = cells[1].text.strip()

        # Check if the value is actually a range such as "1 - 30" or "1-30"
        range_match = re.match(r'^(\d+)\s*[-–]\s*(\d+)$', value_text)
        if range_match:
            start = int(range_match.group(1))
            end = int(range_match.group(2))
            # Add each value in the range
            # This is kind of ugly because we are creating some value lookups
            # that have thousands of values...a function would be better but I
            # was trying to keep the metadata 'pure' data
            for i in range(start, end + 1):
                value_dict[i] = description
        else:
            # Try to parse as single integer
            try:
                value = int(value_text)
                value_dict[value] = description
            except:
                # If not a number, store as None
                value_dict[None] = description

    return value_dict

def parse_codebook_html(html_path: Path) -> Dict[str, ColumnMetadata]:
    """
    Parse the BRFSS codebook HTML file and extract column metadata.

    Args:
        html_path: Path to the HTML codebook file

    Returns:
        Dictionary mapping SAS variable names to ColumnMetadata objects
    """
    with open(html_path, 'r', encoding='windows-1252') as f:
        html_content = f.read()

    soup = BeautifulSoup(html_content, 'html.parser')

    # Find all div elements with class "branch"
    branches = soup.find_all('div', class_='branch')

    # The first one is the Codebook header table which we don't want
    branches = branches[1:]

    metadata_dict = {}

    for branch in branches:
        # Find the table with summary="Procedure Report: Report"
        table = branch.find('table', attrs={'summary': 'Procedure Report: Report'})
        if not table:
            continue

        # Find the first td in the thead > tr
        thead = table.find('thead')
        if not thead:
            continue

        first_tr = thead.find('tr')
        if not first_tr:
            continue

        # Find td with metadata content - may not have all classes
        metadata_cell = None
        for td in first_tr.find_all('td'):
            text = td.get_text()
            if text:
                # Clean text before checking
                text_clean = text.replace('\xa0', ' ')
                if 'Label:' in text_clean and 'SAS Variable Name:' in text_clean:
                    metadata_cell = td
                    break

        if not metadata_cell:
            continue

        cell_text = metadata_cell.get_text()

        # Check if this cell contains column metadata by looking for key fields
        try:
            # Extract fields using regex - handle non-breaking spaces
            cell_text = cell_text.replace('\xa0', ' ')  # Replace non-breaking spaces

            label_match = re.search(r'Label:\s*(.+?)(?=Section\s*Name:|Core\s*Section\s*Number:|$)', cell_text, re.DOTALL)
            section_name_match = re.search(r'Section\s*Name:\s*(.+?)(?=Core\s*Section\s*Number:|Section\s*Number:|$)', cell_text, re.DOTALL)
            # Handle both "Core Section Number" and "Section Number"
            section_number_match = re.search(r'(?:Core\s*)?Section\s*Number:\s*(\d+)', cell_text)
            question_number_match = re.search(r'Question\s*Number:\s*(\d+)', cell_text)
            column_match = re.search(r'Column:\s*(.+?)(?=Type\s*of\s*Variable:|$)', cell_text, re.DOTALL)
            type_match = re.search(r'Type\s*of\s*Variable:\s*(.+?)(?=SAS\s*Variable\s*Name:|$)', cell_text, re.DOTALL)
            sas_name_match = re.search(r'SAS\s*Variable\s*Name:\s*(.+?)(?=Question\s*Prologue:|Question:|$)', cell_text, re.DOTALL)
            prologue_match = re.search(r'Question\s*Prologue:\s*(.+?)(?=Question:|$)', cell_text, re.DOTALL)
            question_match = re.search(r'Question:\s*(.+?)$', cell_text, re.DOTALL)

            # Only require label and SAS variable name
            if label_match and sas_name_match:

                # Clean up the extracted values
                label = label_match.group(1).strip()
                sas_variable_name = sas_name_match.group(1).strip()

                # Extract optional fields
                section_name = section_name_match.group(1).strip() if section_name_match else None
                section_number = int(section_number_match.group(1)) if section_number_match else None
                question_number = int(question_number_match.group(1)) if question_number_match else None
                column = column_match.group(1).strip() if column_match else None
                type_of_variable = type_match.group(1).strip() if type_match else None
                question_prologue = prologue_match.group(1).strip() if prologue_match else None
                question = question_match.group(1).strip() if question_match else None

                # Remove any extra whitespace or newlines
                if question_prologue and not question_prologue:
                    question_prologue = None

                # Create ColumnMetadata object
                metadata = ColumnMetadata(
                    label=label,
                    sas_variable_name=sas_variable_name,
                    section_name=section_name,
                    section_number=section_number,
                    question_number=question_number,
                    column=column,
                    type_of_variable=type_of_variable,
                    question_prologue=question_prologue,
                    question=question,
                    value_lookup=get_value_lookup(table),
                    computed= True if section_name == 'Calculated Variables' or section_name == 'Calculated Race Variables' else False
                )

                metadata_dict[sas_variable_name] = metadata

        except Exception as e:
            # Skip cells that don't parse correctly but show problems
            print(e)

    return metadata_dict


bfrss_metadata = parse_codebook_html(BFRSS_CODEBOOK_PATH)

In [40]:
# Display the number of columns parsed
print(f"Parsed {len(bfrss_metadata)} column definitions from the codebook")

# Show a sample of the metadata
sample_keys = list(bfrss_metadata.keys())[:5]
for key in sample_keys:
    metadata = bfrss_metadata[key]
    print(f"\n{key}:")
    print(f"  Label: {metadata.label}")
    print(f"  Question: {metadata.question}")
    print(f"  Column: {metadata.column}")
    print(f"  Type: {metadata.type_of_variable}")
    print(f"  Computed: {metadata.computed}")
    print(f"  Section Name: {metadata.section_name}")
    print(f"  Section Number: {metadata.section_number}")
    print(f"  Question Number: {metadata.question_number}")

Parsed 344 column definitions from the codebook

_STATE:
  Label: State FIPS Code
  Question: State FIPS Code
  Column: 1-2
  Type: Num
  Computed: False
  Section Name: Record Identification
  Section Number: 0
  Question Number: 1

FMONTH:
  Label: File Month
  Question: File Month
  Column: 17-18
  Type: Num
  Computed: False
  Section Name: Record Identification
  Section Number: 0
  Question Number: 8

IDATE:
  Label: Interview Date
  Question: Interview Date
  Column: 19-26
  Type: Char
  Computed: False
  Section Name: Record Identification
  Section Number: 0
  Question Number: 9

IMONTH:
  Label: Interview Month
  Question: Interview Month
  Column: 19-20
  Type: Char
  Computed: False
  Section Name: Record Identification
  Section Number: 0
  Question Number: 10

IDAY:
  Label: Interview Day
  Question: Interview Day
  Column: 21-22
  Type: Char
  Computed: False
  Section Name: Record Identification
  Section Number: 0
  Question Number: 11


# Metadata Documentation
Notes and examples of the metadata extraction:



In [41]:
print(f"Total columns in dataframe: {len(bfrss_raw_df.columns)}")
print(f"Total metadata parsed: {len(bfrss_metadata)}")
print(f"Coverage: {len(bfrss_metadata) / len(bfrss_raw_df.columns) * 100:.1f}%")

# Check which columns don't have metadata
missing_metadata = [col for col in bfrss_raw_df.columns if col not in bfrss_metadata]
print(f"\nColumns without metadata: {len(missing_metadata)}")
if missing_metadata:
    print("First 10 missing:", missing_metadata[:10])
print("Note: There is data for these columns but no metadata is available, likely purged bc of policy changes.")

Total columns in dataframe: 350
Total metadata parsed: 344
Coverage: 98.3%

Columns without metadata: 7
First 10 missing: ['LNDSXBRT', 'CELSXBRT', 'BIRTHSEX', 'TRNSGNDR', 'USEMRJN4', 'RCSGEND1', 'RCSXBRTH']
Note: There is data for these columns but no metadata is available, likely purged bc of policy changes.


## Understanding the Friendly Mapping Feature
(note I generated the following docs and examples with ChatGPT, but I've vetted all of it)

The metadata parser includes a powerful "friendly mapping" feature that translates numeric codes in the dataset to their human-readable descriptions. This is particularly useful for categorical variables where numeric codes represent specific responses.

### How It Works

Each `ColumnMetadata` object contains a `value_lookup` dictionary that maps numeric values (or None) to their text descriptions. This mapping is automatically extracted from the codebook HTML file during parsing.

#### Key Components:

1. **`value_lookup` dictionary**: Found in each `ColumnMetadata` object
   - Keys: Numeric codes (int) or None
   - Values: Human-readable descriptions (str)

2. **Automatic extraction**: The `get_value_lookup()` function in `parser.py` extracts these mappings from HTML tables in the codebook

### Example 1: Understanding what values mean for a specific column

In [43]:
# Example 1: Understanding what values mean for a specific column
# Let's look at the _STATE column which has distinct state codes

state_metadata = bfrss_metadata['_STATE']
print(f"Column: {state_metadata.sas_variable_name}")
print(f"Label: {state_metadata.label}")
print(f"Question: {state_metadata.question}")
print(f"\nSample of value mappings (first 10):")
# Show first 10 state mappings
for i, (value, description) in enumerate(state_metadata.value_lookup.items()):
    if i < 10:
        print(f"  {value}: {description}")

Column: _STATE
Label: State FIPS Code
Question: State FIPS Code

Sample of value mappings (first 10):
  1: Alabama
  2: Alaska
  4: Arizona
  5: Arkansas
  6: California
  8: Colorado
  9: Connecticut
  10: Delaware
  11: District of Columbia
  12: Florida


### Example 2: Translating values in your data

In [44]:
# Example 2: Translating values in your data
# Let's translate some actual STATE values from the dataframe

# Get a sample of state values
sample_values = bfrss_raw_df['_STATE'].value_counts().head(10)
print("Top 10 states by number of respondents:\n")

for value, count in sample_values.items():
    # Get the description from value_lookup
    description = state_metadata.value_lookup.get(int(value) if not pd.isna(value) else None, "Unknown")
    print(f"Code {int(value)}: {description} (Count: {count:,})")

Top 10 states by number of respondents:

Code 53: Washington (Count: 26,444)
Code 36: New York (Count: 17,349)
Code 24: Maryland (Count: 17,255)
Code 27: Minnesota (Count: 16,170)
Code 39: Ohio (Count: 13,384)
Code 12: Florida (Count: 13,255)
Code 31: Nebraska (Count: 12,886)
Code 55: Wisconsin (Count: 12,819)
Code 23: Maine (Count: 12,255)
Code 4: Arizona (Count: 12,036)


### Example 3: Creating a mapping function for easy translation

In [45]:
# Example 3: Creating a mapping function for easy translation
def translate_column_values(df, column_name, metadata_dict):
    """
    Translate numeric codes to descriptions for a specific column.

    Args:
        df: The dataframe containing the data
        column_name: Name of the column to translate
        metadata_dict: Dictionary of column metadata

    Returns:
        Pandas Series with translated values
    """
    if column_name not in metadata_dict:
        print(f"No metadata found for column: {column_name}")
        return df[column_name]

    metadata = metadata_dict[column_name]

    # Create translation function
    def translate(value):
        if pd.isna(value):
            return "Missing"
        return metadata.value_lookup.get(int(value), f"Unknown code: {value}")

    return df[column_name].apply(translate)

# Example usage - translate STATE codes
bfrss_raw_df['STATE_NAME'] = translate_column_values(bfrss_raw_df, '_STATE', bfrss_metadata)

# Show sample
print("Sample of translated state values:")
print(bfrss_raw_df[['_STATE', 'STATE_NAME']].head(10))

Sample of translated state values:
   _STATE STATE_NAME
0     1.0    Alabama
1     1.0    Alabama
2     1.0    Alabama
3     1.0    Alabama
4     1.0    Alabama
5     1.0    Alabama
6     1.0    Alabama
7     1.0    Alabama
8     1.0    Alabama
9     1.0    Alabama


### Example 4: Working with columns that have ranges

In [46]:
# Example 4: Working with columns that have ranges
# Now let's test with POORHLTH which has a range value "1 - 30"
poorhlth_metadata = bfrss_metadata['POORHLTH']
print(f"Column: {poorhlth_metadata.sas_variable_name}")
print(f"Label: {poorhlth_metadata.label}")

# Check if the range was properly expanded
print(f"\nTotal value mappings: {len(poorhlth_metadata.value_lookup)}")
print("\nSample mappings:")
# Show some specific values to verify range expansion
for value in [1, 15, 30, 77, 88, 99]:
    if value in poorhlth_metadata.value_lookup:
        print(f"  {value}: {poorhlth_metadata.value_lookup[value]}")

Column: POORHLTH
Label: Poor Physical or Mental Health

Total value mappings: 34

Sample mappings:
  1: Number of daysNotes: _ _ Number of days
  15: Number of daysNotes: _ _ Number of days
  30: Number of daysNotes: _ _ Number of days
  77: Don't know/Not sure
  88: None
  99: Refused


### Example 5: Batch translation of multiple columns

In [47]:
# Example 5: Batch translation of multiple columns
# This example shows how to efficiently translate multiple columns at once

def batch_translate_columns(df, column_list, metadata_dict):
    """
    Translate multiple columns from numeric codes to descriptions.

    Args:
        df: The dataframe containing the data
        column_list: List of column names to translate
        metadata_dict: Dictionary of column metadata

    Returns:
        Dictionary of translated series
    """
    translated = {}

    for col in column_list:
        if col in metadata_dict and col in df.columns:
            translated[f"{col}_DESC"] = translate_column_values(df, col, metadata_dict)
            print(f"Translated {col}")
        else:
            print(f"Skipped {col} (not found in metadata or dataframe)")

    return translated

# Translate several categorical columns
columns_to_translate = ['_STATE', 'FMONTH', 'DISPCODE', 'SEX1']
translations = batch_translate_columns(bfrss_raw_df, columns_to_translate, bfrss_metadata)

# Add translations to dataframe
for col_name, translated_series in translations.items():
    bfrss_raw_df[col_name] = translated_series

# Show sample of multiple translations
print("\nSample of translated data:")
original_cols = columns_to_translate[:3]  # Show first 3
desc_cols = [f"{col}_DESC" for col in original_cols]
print(bfrss_raw_df[original_cols + desc_cols].head())

Translated _STATE
Translated FMONTH
Translated DISPCODE
Skipped SEX1 (not found in metadata or dataframe)

Sample of translated data:
   _STATE  FMONTH  DISPCODE _STATE_DESC FMONTH_DESC        DISPCODE_DESC
0     1.0     1.0    1100.0     Alabama     January  Completed Interview
1     1.0     1.0    1100.0     Alabama     January  Completed Interview
2     1.0     1.0    1100.0     Alabama     January  Completed Interview
3     1.0     1.0    1100.0     Alabama     January  Completed Interview
4     1.0     1.0    1100.0     Alabama     January  Completed Interview


### Binning by Section Name